<a href="https://colab.research.google.com/github/Niharika-Ravela/Niharika_INFO5502_Summer2022/blob/main/lab_assignment_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The third Lab-assignment (07/22/2022 11:59'AM' - 07/26/2022 11:59PM, 50 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points). Fomulate your domain problem: Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [1]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
I am considering the IMDb website to get the Top 1000 movies in the year 2021 from the website. The result is to get the top rated films.
To answer this question we have to scrape the data from the 'https://www.imdb.com/search/title/?release_date=2021-01-01,2021-12-31&sort=boxoffice_gross_us,desc&start=' 

For answering this kind of problem we need to scrape the raw data from the direct website. For my problem I can use 1000 data points for the analysis.
The steps that I took are:
1. First, to scrape the data we need to have a library called BeautifulSoup and requests.
2. After that we need to give the URL and store it in a variable so that we can parse the data.
3. After saving the url of the website we need to request the website for the data using the requests library.
4. Next, Use the BeautifulSoup to get the HTML i.e., the raw data from the website.
5. To get the exact tag that we want we have to use Soup.findall method
6. Finally, we have to remove all HTML tags and parse individually and convert into plain text.

In particular in this case since I was using the IMDb Website not only this one but every site can have multiple pages to extract data from multiple pages we need to first
use a base by getting one data point in this case also I first tried to get the first top movie by its all details.
Next, I tried creating functions that iterate through the pages i.e., it can happend by finding the element from the inspect part of the website we can get the exact element we are looking for.
Once I got the data points I stored them in the dataframe. But while iterating I used some counters to keep track of the page numbers and the number of titles that are being collected.


'''

"\nPlease write you answer here:\nI am considering the IMDb website to get the Top 1000 movies in the year 2021 from the website. The result is to get the top rated films.\nTo answer this question we have to scrape the data from the 'https://www.imdb.com/search/title/?release_date=2021-01-01,2021-12-31&sort=boxoffice_gross_us,desc&start=' \n\nFor answering this kind of problem we need to scrape the raw data from the direct website. For my problem I can use 1000 data points for the analysis.\nThe steps that I took are:\n1. First, to scrape the data we need to have a library called BeautifulSoup and requests.\n2. After that we need to give the URL and store it in a variable so that we can parse the data.\n3. After saving the url of the website we need to request the website for the data using the requests library.\n4. Next, Use the BeautifulSoup to get the HTML i.e., the raw data from the website.\n5. To get the exact tag that we want we have to use Soup.findall method\n6. Finally, we ha

Question 2 (30 points). Collect your data to answer the research problem: Write python code to collect 1000 data samples you discussed above.

In [2]:
!pip install translate
!pip install googletrans==3.1.0a0

import bs4
import requests
import time
import random as ran
import sys
import pandas as pd

from googletrans import Translator
translator = Translator()


In [3]:
url = 'https://www.imdb.com/search/title/?release_date=2021-01-01,2021-12-31&sort=boxoffice_gross_us,desc&start='

source = requests.get(url).text
soup = bs4.BeautifulSoup(source,'html.parser')

movie_box = soup.findAll('div',{'class':'lister-item-content'})
#print(movie_box)

mov_name = movie_box[0].find('a').get_text() # Name of the movie
result = translator.translate(mov_name, dest = 'en')
mov_name = result.text
m_reyear = int(movie_box[0].find('span',{'class': 'lister-item-year'}).contents[0][1:-1]) # Release year

m_rating = float(movie_box[0].find('div',{'class':'inline-block ratings-imdb-rating'}).get('data-value')) #rating

m_mscore = float(movie_box[0].find('span',{'class':'metascore favorable'}).contents[0].strip()) #meta score

m_votes = int(movie_box[0].find('span',{'name':'nv'}).get('data-value')) # votes

print("Movie Name: " + mov_name,
      "\nRelease Year: " + str(m_reyear),
      "\nIMDb Rating: " + str(m_rating),
      "\nMeta score: " + str(m_mscore),
      "\nVotes: " + '{:,}'.format(m_votes)

)  # For just one instance


#For all the multiple pages coding

def scrape_mblock(movie_block):
    
    movieb_data ={}
  
    try:
        movieb_data['name'] = movie_block.find('a').get_text() # Name of the movie
        result = translator.translate(movieb_data['name'], dest = 'en')
        movieb_data['name'] = result.text
    except:
        movieb_data['name'] = None


    return movieb_data



def scrape_m_page(movie_box):
    
    page_movie_data = []
    num_blocks = len(movie_box)
    
    for block in range(num_blocks):
        page_movie_data.append(scrape_mblock(movie_box[block]))
    
    return page_movie_data


def scrape_this(link,t_count):

    base_url = link
    target = t_count
    
    start = 0
    stop = 0
    left = target - stop 
    
    page_no = 1
    
    movie_data = []
    
    
    while left > 0:

        url = base_url + str(page_no)
        source = requests.get(url).text
        soup = bs4.BeautifulSoup(source,'html.parser')
        
        movie_box = soup.findAll('div',{'class':'lister-item-content'})
        
        movie_data.extend(scrape_m_page(movie_box))   
        
        start = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[0])

        stop = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[1].split(" ")[0])

        left = target - stop
        
        print('\r' + "currently scraping movies from: " + str(start) + " - "+str(stop), "| remaining count: " + str(left), flush=True, end ="")
        
        page_no = stop + 1
           
        time.sleep(ran.randint(0, 10))
    
    return movie_data


base_scraping_link = "https://www.imdb.com/search/title/?release_date=2021-01-01,2021-12-31&sort=boxoffice_gross_us,desc&start="

top_movies = 1000 
films = []

movies = scrape_this(base_scraping_link,950)

print('\r'+"List of top " + str(top_movies) +" movies:" + "\n", end="\n")
movies=pd.DataFrame(movies)
print(movies)



Movie Name: Spider-Man: No Way Home 
Release Year: 2021 
IMDb Rating: 8.3 
Meta score: 71.0 
Votes: 688,957
List of top 1000 movies:s from: 901 - 950 | remaining count: 000

                                          name
0                      Spider-Man: No Way Home
1    Shang-Chi and the Legend of the Ten Rings
2                  Venom: Let There Be Carnage
3                                  Black Widow
4                            F9: The Fast Saga
..                                         ...
944                                Do, Re & Mi
945                                Do, Re & Mi
946                                Do, Re & Mi
947                                Do, Re & Mi
948                                Do, Re & Mi

[949 rows x 1 columns]


In [6]:
display(movies.to_string())

"                                                          name\n0                                      Spider-Man: No Way Home\n1                    Shang-Chi and the Legend of the Ten Rings\n2                                  Venom: Let There Be Carnage\n3                                                  Black Widow\n4                                            F9: The Fast Saga\n5                                                     Eternals\n6                                                       Sing 2\n7                                               No Time to Die\n8                                                     Free Guy\n9                                                         Dune\n10                                           Godzilla vs. Kong\n11                                                       Charm\n12                                                     Cruella\n13                                           The Forever Purge\n14                                     

Question 3 (10 points). Understand the data quality: Search a second hand dataset (any dataset) from kaggle or other websites. Describe the data quality problem of the dataset and explain your strtegy to clean the data.

In [4]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):
'''

Please write you answer here:

From Kaggle I have considered a dataset called:Customer Personality Analysis. 
Customer personality analysis is a detailed analysis of a company's ideal customers. 
This helps companies better understand their customers and makes it easier  to tailor their products  to the specific needs, behaviors, and concerns of different types of customers. 
Customer personality analysis helps companies modify their products based on  target customers from different types of customer segments. 
For example, instead of spending money to sell a new product to all customers in the company's database, the company analyzes which customer segment is most likely to buy the product and  only to that particular segment. You can sell the product.
From the dataset there are some features that needs a little work. 
Apart from that there are also missing values that can be replaced by the either imputyation or interpolation but in this case we can use imputation.
For that first we need to find the analysis of the feature depending on that we can use either mean or mode or median values to replace the missing values or some cases we can simply replace them by NaN. 
After replacing all the missing values then we need to check if all the features are usefull or not.
In this case I can tellm that for 'Maritial_Status' There are marrie, divoreced, single and two ohter types-YOLO and Absurd. In my opinion we can just consider them as the single type.
Not only this there are also few variable that count the Amt individually for each category we can just combine all of them into one variable mentioning the Amount spent.
By doing these steps we can make sure that the data is clean and we can proceed to the next steps by performing the data analysis and forming research questions.

'''

"\n\nPlease write you answer here:\n\nFrom Kaggle I have considered a dataset called:Customer Personality Analysis. \nCustomer personality analysis is a detailed analysis of a company's ideal customers. \nThis helps companies better understand their customers and makes it easier  to tailor their products  to the specific needs, behaviors, and concerns of different types of customers. \nCustomer personality analysis helps companies modify their products based on  target customers from different types of customer segments. \nFor example, instead of spending money to sell a new product to all customers in the company's database, the company analyzes which customer segment is most likely to buy the product and  only to that particular segment. You can sell the product.\nFrom the dataset there are some features that needs a little work. \nApart from that there are also missing values that can be replaced by the either imputyation or interpolation but in this case we can use imputation.\nFor